In [1]:
# import libraries
import minsearch
import json
from groq import Groq
from dotenv import load_dotenv
import os

In [2]:
# load data
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

# load environment variables
load_dotenv()

# create client calling Groq class
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

### Min Search

In [ ]:
# flatten data
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
def index_documents(documents):
    # create the index
    index = minsearch.Index(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"]
    )

    # add the documents to the index
    index.fit(documents)
    return index

In [4]:
def search(query, index):
    boost = {'question': 5.0, 'section': 0.5}
    section = {'course': 'data-engineering-zoomcamp'}

    results = index.search(
        query = query,
        filter_dict= section,
        boost_dict = boost,
        num_results = 6
    )
    return results

In [5]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant,  answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't have the answer output NONE.
    Make your output concise and dont mention what you found or you didn't in the CONTEXT only.
    Avoid phrases like according the context, based on the context, etc.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [6]:
def llm(prompt):
    # create a query
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )
    # print the response
    return response.choices[0].message.content

In [7]:
def rag(query):
    index = index_documents(documents)
    search_results = search(query, index)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [8]:
print(rag("Can i get certificate if i do the course in self paced mode?"))

No, you can only get a certificate if you finish the course with a “live” cohort.


### Elastic Search

In [10]:
from elasticsearch import Elasticsearch

In [11]:
es_client = Elasticsearch('http://localhost:9200')

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
from tqdm.auto import tqdm

In [14]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [15]:
query = 'I just disovered the course. Can I still join it?'

In [23]:
def elastic_search(query):
    search_query = {
        "size": 6,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [24]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [25]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
print(rag(query))

Yes, you can still join the course even after the start date.
